In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
import talib
import os
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv, find_dotenv
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

In [2]:
load_dotenv()

True

In [3]:
ticker = ["SPY"]
# Set timeframe to "1Day" for Alpaca API
timeframe = "1day"
# Format current date as ISO format
start_date = pd.Timestamp("2019-01-06", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2023-01-28", tz="America/New_York").isoformat()
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2"
)


stock_data = api.get_bars(
    ticker,
    timeframe,
    start=start_date,
    end=end_date
).df

# Reorganize the DataFrame
# Separate ticker data
# Drop unwanted coloumns - Keep  Close
stock_data.drop(columns= ['trade_count','vwap','symbol'], inplace= True)
    # Combine individual stocks into a single data frame

  # Rename the coloumns to the ticker symbols in the list
#stock_data.columns = ticker
stock_data.index.name = 'Date'
stock_data.index_column = 'Date'
# Drop the N/As


# Drop the time component of the date
#df_ticker.index = df_ticker.index.date


# Display the data
display(stock_data)

,open,high,low,close,volume
Date,,,,,
2019-01-07 05:00:00+00:00,252.700,255.9500,251.69,254.29,103705813
2019-01-08 05:00:00+00:00,256.820,257.3100,254.00,256.62,103267687
2019-01-09 05:00:00+00:00,257.560,258.9100,256.19,257.92,95741225
2019-01-10 05:00:00+00:00,256.260,259.1600,255.50,258.98,98012473
2019-01-11 05:00:00+00:00,257.680,259.0098,257.03,258.81,74684741
...,...,...,...,...,...
2023-01-23 05:00:00+00:00,396.720,402.6450,395.72,400.63,84187957
2023-01-24 05:00:00+00:00,398.880,401.1500,397.64,400.20,59549719
2023-01-25 05:00:00+00:00,395.950,400.7000,393.56,400.35,84826825


In [4]:
# Calculate the 8-day and 20-day EMAs
stock_data["ema_8"] = stock_data["close"].ewm(span=8, adjust=False).mean()
stock_data["ema_20"] = stock_data["close"].ewm(span=20, adjust=False).mean()

In [5]:
# Create a new column indicating the direction of the crossover/crossunder
stock_data["crossover"] = np.where(stock_data["ema_8"] > stock_data["ema_20"], 1, 0)
stock_data["crossunder"] = np.where(stock_data["ema_8"] < stock_data["ema_20"], -1, 0)

In [6]:
# Replace 0 values in the crossunder column with the value from the crossover column
stock_data["crossover_crossunder"] = np.where(stock_data["crossunder"] == 0, stock_data["crossover"], stock_data["crossunder"])

In [7]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(stock_data[["ema_8", "ema_20"]], stock_data["crossover_crossunder"], test_size=0.2, random_state=0)


In [8]:
# Define the ANN model
model = Sequential()
model.add(Dense(8, input_dim=2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [9]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [10]:
# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=10)

Epoch 1/400
82/82 [==============================] - 0s 743us/step - loss: 69.8350 - accuracy: 0.0012
Epoch 2/400
82/82 [==============================] - 0s 703us/step - loss: 8.3376 - accuracy: 0.0697
Epoch 3/400
82/82 [==============================] - 0s 728us/step - loss: 4.9091 - accuracy: 0.1039
Epoch 4/400
82/82 [==============================] - 0s 723us/step - loss: 4.9428 - accuracy: 0.1357
Epoch 5/400
82/82 [==============================] - 0s 709us/step - loss: 4.8293 - accuracy: 0.0941
Epoch 6/400
82/82 [==============================] - 0s 710us/step - loss: 4.6327 - accuracy: 0.0868
Epoch 7/400
82/82 [==============================] - 0s 722us/step - loss: 4.7040 - accuracy: 0.1051
Epoch 8/400
82/82 [==============================] - 0s 710us/step - loss: 4.6114 - accuracy: 0.0746
Epoch 9/400
82/82 [==============================] - 0s 723us/step - loss: 4.7758 - accuracy: 0.1467
Epoch 10/400
82/82 [==============================] - 0s 729us/step - loss: 4.5110 - accur

In [11]:
# Evaluate the model on the test set
scores = model.evaluate(X_test, y_test)


7/7 [==============================] - 0s 1ms/step - loss: -7.1371 - accuracy: 0.6146


In [12]:
# Print the accuracy of the model on the test set
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 61.46%
